In [81]:
import pandas as pd


places_data = pd.read_csv("final-dataset/main_dataset.csv") 
reviews_data = pd.read_csv("final-dataset/review_dataset.csv")  

user_place_reviews = pd.merge(reviews_data,places_data, on='id')

# drop review_number, lat long, address, url,status, phone, types_y, price-level,review 1-5
user_place_reviews = user_place_reviews.drop(columns=['review_number', 'latitude', 'longitude', 'address', 'url', 'status', 'phone', 'types_y', 'price-level', 'review 1', 'review 2', 'review 3', 'review 4', 'review 5'])
user_place_reviews.head()

,id,types_x,review,user_id,sentiment-score,name,primary-type,rating,rating-count
0,ChIJYcGr7GSb0S0RckePBrCWikw,"hotel, lodging","It has quite small room, and the hallway is qu...",user_2573,0.506250,Singaraja Hotel (ex- POP! Hotel Hardys Singara...,hotel,4.1,2581.0
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,"indonesian_restaurant, restaurant, food","Surprisingly, a really good warung that’s hidd...",user_2062,0.606250,Warung Bik Juk,indonesian_restaurant,4.5,648.0
2,ChIJYyHbhgia0S0RzdjNXLmcf54,"tourist_attraction, restaurant, food","Only had a fleeting visit here, came by coach,...",user_2348,0.491667,Harbour Tourist Area of Buleleng,tourist_attraction,4.3,2800.0
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,"coffee_shop, cafe, food, store","One word, underrated! How come place like this...",user_1448,0.672338,Abuela Coffee,coffee_shop,4.9,164.0
4,ChIJxaITmQia0S0RyrbukE8vsJU,"tourist_attraction, place_of_worship","This temple is located in Singaraja, located i...",user_4426,0.569762,Klenteng Ling Gwan Kiong,place_of_worship,4.6,142.0


In [82]:
from sklearn.preprocessing import MultiLabelBinarizer

user_place_reviews['types_x'] = user_place_reviews['types_x'].str.split(', ')

mlb = MultiLabelBinarizer()
one_hot = mlb.fit_transform(user_place_reviews['types_x'])

one_hot_df = pd.DataFrame(one_hot, columns=mlb.classes_)

user_place_reviews = pd.concat([user_place_reviews, one_hot_df], axis=1)

user_place_reviews = user_place_reviews.drop_duplicates(subset=['user_id', 'id'])

# user_place_reviews = user_place_reviews.drop(columns=['types_x'])

user_place_reviews.head()

,id,types_x,review,user_id,sentiment-score,name,primary-type,rating,rating-count,airport,...,tourist_attraction,travel_agency,turkish_restaurant,university,vegan_restaurant,vegetarian_restaurant,vietnamese_restaurant,wedding_venue,wholesaler,zoo
0,ChIJYcGr7GSb0S0RckePBrCWikw,"[hotel, lodging]","It has quite small room, and the hallway is qu...",user_2573,0.506250,Singaraja Hotel (ex- POP! Hotel Hardys Singara...,hotel,4.1,2581.0,0,...,0,0,0,0,0,0,0,0,0,0
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,"[indonesian_restaurant, restaurant, food]","Surprisingly, a really good warung that’s hidd...",user_2062,0.606250,Warung Bik Juk,indonesian_restaurant,4.5,648.0,0,...,0,0,0,0,0,0,0,0,0,0
2,ChIJYyHbhgia0S0RzdjNXLmcf54,"[tourist_attraction, restaurant, food]","Only had a fleeting visit here, came by coach,...",user_2348,0.491667,Harbour Tourist Area of Buleleng,tourist_attraction,4.3,2800.0,0,...,1,0,0,0,0,0,0,0,0,0
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,"[coffee_shop, cafe, food, store]","One word, underrated! How come place like this...",user_1448,0.672338,Abuela Coffee,coffee_shop,4.9,164.0,0,...,0,0,0,0,0,0,0,0,0,0
4,ChIJxaITmQia0S0RyrbukE8vsJU,"[tourist_attraction, place_of_worship]","This temple is located in Singaraja, located i...",user_4426,0.569762,Klenteng Ling Gwan Kiong,place_of_worship,4.6,142.0,0,...,1,0,0,0,0,0,0,0,0,0


In [83]:
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# List of place type columns
place_types = [
    'airport', 'american_restaurant', 'amusement_center', 'amusement_park', 'art_gallery',
    'athletic_field', 'atm', 'bakery', 'banquet_hall', 'bar', 'barbecue_restaurant',
    'barber_shop', 'beauty_salon', 'bed_and_breakfast', 'bicycle_store', 'brazilian_restaurant',
    'breakfast_restaurant', 'brunch_restaurant', 'cafe', 'campground', 'camping_cabin',
    'car_rental', 'car_wash', 'cemetery', 'child_care_agency', 'chinese_restaurant', 'church',
    'clothing_store', 'coffee_shop', 'community_center', 'consultant', 'convention_center', 'cottage',
    'cultural_center', 'dog_park', 'event_venue', 'extended_stay_hotel', 'farm', 'farmstay',
    'fast_food_restaurant', 'finance', 'fitness_center', 'florist', 'food', 'french_restaurant',
    'furniture_store', 'general_contractor', 'gift_shop', 'golf_course', 'greek_restaurant',
    'grocery_store', 'guest_house', 'gym', 'hair_care', 'hair_salon', 'hamburger_restaurant',
    'health', 'heliport', 'hiking_area', 'hindu_temple', 'historical_landmark', 'home_goods_store',
    'home_improvement_store', 'hostel', 'hotel', 'ice_cream_shop', 'indian_restaurant',
    'indonesian_restaurant', 'italian_restaurant', 'japanese_restaurant', 'jewelry_store',
    'korean_restaurant', 'lebanese_restaurant', 'library', 'liquor_store', 'lodging', 'market',
    'meal_delivery', 'meal_takeaway', 'mediterranean_restaurant', 'mexican_restaurant',
    'middle_eastern_restaurant', 'mosque', 'motel', 'movie_theater', 'museum', 'national_park',
    'night_club', 'park', 'performing_arts_theater', 'pizza_restaurant',
    'place_of_worship', 'playground', 'preschool', 'private_guest_room', 'ramen_restaurant',
    'real_estate_agency', 'resort_hotel', 'rest_stop', 'restaurant', 'sandwich_shop', 'school',
    'seafood_restaurant', 'shopping_mall', 'spa', 'spanish_restaurant', 'sporting_goods_store',
    'sports_club', 'sports_complex', 'steak_house', 'store', 'supermarket', 'sushi_restaurant',
    'swimming_pool', 'thai_restaurant', 'tourist_attraction', 'travel_agency', 'turkish_restaurant',
    'university', 'vegan_restaurant', 'vegetarian_restaurant', 'vietnamese_restaurant',
    'wedding_venue', 'wholesaler', 'zoo', 'parking'
]


# Sum the place type columns for each user to get their preferences
user_preferences = user_place_reviews.groupby('user_id')[place_types].sum().reset_index()

# Normalize the preferences
user_preferences[place_types] = user_preferences[place_types].div(user_preferences[place_types].sum(axis=1), axis=0)

# Replace non zero values with 1
user_preferences[place_types] = user_preferences[place_types].applymap(lambda x: 1 if x > 0 else 0)

user_preferences.head()


/tmp/ipykernel_1740/2597246789.py:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  user_preferences[place_types] = user_preferences[place_types].applymap(lambda x: 1 if x > 0 else 0)


,user_id,airport,american_restaurant,amusement_center,amusement_park,art_gallery,athletic_field,atm,bakery,banquet_hall,...,travel_agency,turkish_restaurant,university,vegan_restaurant,vegetarian_restaurant,vietnamese_restaurant,wedding_venue,wholesaler,zoo,parking
0,user_1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,user_10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,user_100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,user_1000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,user_1001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate
from sklearn.preprocessing import OneHotEncoder
import numpy as np

reviews = user_place_reviews

# Normalize sentiment-score to a 1-5 scale
reviews['normalized_sentiment'] = 1 + 4 * (reviews['sentiment-score'] - reviews['sentiment-score'].min()) / (reviews['sentiment-score'].max() - reviews['sentiment-score'].min())

# Merge datasets on user_id
merged_data = pd.merge(user_preferences, reviews, on='user_id', how='left')

# Fill missing values in reviews with default values
merged_data.fillna({'normalized_sentiment': 3.0, 'primary-type': 'unknown'}, inplace=True)

# One-hot encode user preferences
user_pref_cols = user_preferences.columns[2:]  # Assuming first two columns are index and user_id
onehot_encoder = OneHotEncoder(sparse_output=False)
user_pref_encoded = onehot_encoder.fit_transform(user_preferences[user_pref_cols])

# One-hot encode primary-type in reviews
type_onehot_encoder = OneHotEncoder(sparse_output=False)
type_encoded = type_onehot_encoder.fit_transform(merged_data[['primary-type']])

# Define the input layers
user_input = Input(shape=(user_pref_encoded.shape[1],), name='user_preferences')
type_input = Input(shape=(type_encoded.shape[1],), name='primary_type')

# Concatenate the inputs
concat = concatenate([user_input, type_input], name='concatenate')

# Add dense layers
dense1 = Dense(128, activation='relu', name='dense1')(concat)
dense2 = Dense(64, activation='relu', name='dense2')(dense1)
output = Dense(type_encoded.shape[1], activation='softmax', name='output')(dense2)

# Build the model
model = Model(inputs=[user_input, type_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Prepare input data for training
user_ids = merged_data['user_id']
user_pref_encoded_dict = dict(zip(user_preferences['user_id'], user_pref_encoded))
user_pref_encoded_list = np.array([user_pref_encoded_dict[uid] for uid in user_ids])

# Train the model
history = model.fit([user_pref_encoded_list, type_encoded], type_encoded,epochs=10, batch_size=32, validation_split=0.2)

# Function to recommend places based on predicted probabilities
def recommend(type_ids, top_n=10):
    type_codes = type_onehot_encoder.transform(np.array(type_ids).reshape(-1, 1))
    dummy_user_pref = np.zeros((type_codes.shape[0], user_pref_encoded.shape[1]))  # Dummy user preferences
    predictions = model.predict([dummy_user_pref, type_codes])
    
    # Get original type names
    type_names = type_onehot_encoder.get_feature_names_out(['primary-type'])
    
    # Create DataFrame to store probabilities
    prob_df = pd.DataFrame(predictions, columns=type_names)
    
    # Sort and get top_n recommendations
    top_recommendations = prob_df.sort_values(by=type_ids, axis=1, ascending=False).iloc[:, :top_n]
    
    # Extract place IDs
    place_ids = top_recommendations.columns.tolist()
    
    return place_ids

# Example: Recommend based on multiple types
type_ids = ['restaurant', 'hotel']
recommended_places = recommend(type_ids)
print(f"Top recommended places based on types {type_ids}: {recommended_places}")


Epoch 1/10
773/773 [==============================] - 9s 10ms/step - loss: 1.3044 - val_loss: 0.3775
Epoch 2/10
773/773 [==============================] - 7s 9ms/step - loss: 0.1922 - val_loss: 0.1057
Epoch 3/10
773/773 [==============================] - 7s 9ms/step - loss: 0.0567 - val_loss: 0.0329
Epoch 4/10
773/773 [==============================] - 8s 10ms/step - loss: 0.0199 - val_loss: 0.0154
Epoch 5/10
773/773 [==============================] - 8s 10ms/step - loss: 0.0085 - val_loss: 0.0084
Epoch 6/10
773/773 [==============================] - 7s 10ms/step - loss: 0.0039 - val_loss: 0.0049
Epoch 7/10
773/773 [==============================] - 7s 9ms/step - loss: 0.0017 - val_loss: 0.0026
Epoch 8/10
773/773 [==============================] - 7s 9ms/step - loss: 9.6715e-04 - val_loss: 0.0015
Epoch 9/10
773/773 [==============================] - 7s 9ms/step - loss: 5.1414e-04 - val_loss: 0.0011
Epoch 10/10
1/1 [==============================] - 0s 78ms/step


/home/tf-gpu/anaconda3/envs/tf-gpu-peko/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


KeyError: 'restaurant'

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

# Hyperparameters
embedding_dim = 50

# User model
user_input = Input(shape=(len(place_types),), name='user_input')
user_embedding = Dense(embedding_dim, activation='relu')(user_input)

# Place model
place_input = Input(shape=(len(place_types),), name='place_input')
place_embedding = Dense(embedding_dim, activation='relu')(place_input)

# Concatenate user and place embeddings
merged = Concatenate()([user_embedding, place_embedding])
dense_1 = Dense(128, activation='relu')(merged)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(1, activation='sigmoid')(dense_2)

# Create and compile the model
model = Model(inputs=[user_input, place_input], outputs=output)
model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 126)]                0         []                            
                                                                                                  
 place_input (InputLayer)    [(None, 126)]                0         []                            
                                                                                                  
 dense (Dense)               (None, 50)                   6350      ['user_input[0][0]']          
                                                                                                  
 dense_1 (Dense)             (None, 50)                   6350      ['place_input[0][0]']         
                                                                                            

In [34]:
# Ensure user_preferences and data are correctly aligned and split the data accordingly
user_preferences = user_place_reviews.groupby('user_id')[place_types].sum().reset_index()
user_preferences[place_types] = user_preferences[place_types].div(user_preferences[place_types].sum(axis=1), axis=0)

# Merge data with user preferences to align place features with user preferences
merged_data = user_place_reviews.merge(user_preferences, on='user_id', suffixes=('', '_user'))

# Prepare user features and place features for each review
user_features = merged_data[[f'{ptype}_user' for ptype in place_types]].values
place_features = merged_data[place_types].values

# Generate labels (assuming binary relevance for simplicity)
# You need to ensure your labels are the same length as user and place features
labels = np.random.randint(2, size=(len(user_features),))

# Split into train and test sets ensuring the lengths match
user_train, user_test, place_train, place_test, y_train, y_test = train_test_split(
    user_features, place_features, labels, test_size=0.3, random_state=42
)


In [35]:
# Fit the model
history = model.fit(
    [user_train, place_train],
    y_train,
    epochs=15,
    batch_size=32,
    validation_data=([user_test, place_test], y_test)
)


Epoch 1/15
676/676 [==============================] - 10s 12ms/step - loss: 0.6932 - accuracy: 0.5037 - val_loss: 0.6937 - val_accuracy: 0.4989
Epoch 2/15
676/676 [==============================] - 8s 12ms/step - loss: 0.6932 - accuracy: 0.4999 - val_loss: 0.6933 - val_accuracy: 0.4927
Epoch 3/15
676/676 [==============================] - 8s 12ms/step - loss: 0.6931 - accuracy: 0.5027 - val_loss: 0.6932 - val_accuracy: 0.4987
Epoch 4/15
676/676 [==============================] - 8s 12ms/step - loss: 0.6930 - accuracy: 0.5079 - val_loss: 0.6935 - val_accuracy: 0.4996
Epoch 5/15
676/676 [==============================] - 11s 16ms/step - loss: 0.6929 - accuracy: 0.5092 - val_loss: 0.6932 - val_accuracy: 0.5022
Epoch 6/15
676/676 [==============================] - 10s 15ms/step - loss: 0.6929 - accuracy: 0.5056 - val_loss: 0.6934 - val_accuracy: 0.4987
Epoch 7/15
676/676 [==============================] - 12s 18ms/step - loss: 0.6929 - accuracy: 0.5081 - val_loss: 0.6933 - val_accuracy: 0.

In [36]:
def recommend_places(user_id, top_n=5):
    user_pref = user_preferences[user_preferences['user_id'] == user_id][place_types].values
    place_features = user_place_reviews[place_types].drop_duplicates().values
    place_ids = user_place_reviews['id'].drop_duplicates().values

    predictions = model.predict([np.repeat(user_pref, len(place_features), axis=0), place_features])
    top_indices = np.argsort(predictions[:, 0])[-top_n:][::-1]

    recommended_place_ids = place_ids[top_indices]
    
    # Ensure unique recommendations by using a set
    unique_recommendations = set(recommended_place_ids)
    
    # Retrieve details for unique recommendations
    recommended_places = user_place_reviews[user_place_reviews['id'].isin(unique_recommendations)]
    sorted_recommendations = recommended_places.sort_values(by='rating', ascending=False)
    sorted_recommendations = sorted_recommendations.drop_duplicates(subset=['name'])
    
    return sorted_recommendations[['name', 'primary-type', 'rating']]


In [37]:
user_id = 'user_10'
recommendations = recommend_places(user_id)

print("Top Recommendations for User:", user_id)
print("---------------------------------")
for index, row in recommendations.iterrows():
    print(f"Name: {row['name']}")
    print(f"Type: {row['primary-type']}")
    print(f"Rating: {row['rating']}")
    print("---------------------------------")


22/22 [==============================] - 0s 5ms/step
Top Recommendations for User: user_10
---------------------------------
Name: Melanting waterfall
Type: tourist_attraction
Rating: 4.7
---------------------------------
Name: Babi Guling Sunari
Type: indonesian_restaurant
Rating: 4.5
---------------------------------
Name: Warung Nyoman
Type: indonesian_restaurant
Rating: 4.1
---------------------------------
Name: Mutiara Resto Kintamani
Type: restaurant
Rating: 4.0
---------------------------------
Name: Hotel Pancasari
Type: hotel
Rating: 3.8
---------------------------------


In [38]:
model.save('user_pref_model.keras')

In [39]:
import numpy as np

# Function to recommend places based on user preferences of types
def recommend_places_by_types(user_id, top_n=5):
    # Assuming user preferences are one-hot encoded and stored in user_preferences DataFrame
    user_pref = user_preferences[user_preferences['user_id'] == user_id][place_types].values
    place_features = user_place_reviews[place_types].drop_duplicates().values
    place_ids = user_place_reviews['id'].drop_duplicates().values
    
    # Calculate relevance score based on user preferences and place features
    relevance_scores = np.dot(place_features, user_pref.T).flatten()
    
    # Get indices of top places based on relevance score
    top_indices = np.argsort(relevance_scores)[-top_n:][::-1]
    
    # Get recommended place IDs
    recommended_place_ids = place_ids[top_indices]
    
    # Retrieve details of recommended places
    recommended_places = user_place_reviews[user_place_reviews['id'].isin(recommended_place_ids)]
    sorted_recommendations = recommended_places.sort_values(by='rating', ascending=False)
    sorted_recommendations = sorted_recommendations.drop_duplicates(subset=['name'])
    
    return sorted_recommendations[['name', 'primary-type', 'rating']]

# Example usage
user_id = 'user_10'
recommendations = recommend_places_by_types(user_id)
print("Top Recommendations for User:", user_id)
print("---------------------------------")
for index, row in recommendations.iterrows():
    print(f"Name: {row['name']}")
    print(f"Type: {row['primary-type']}")
    print(f"Rating: {row['rating']}")
    print("---------------------------------")


Top Recommendations for User: user_10
---------------------------------
Name: Warung Jojo's
Type: restaurant
Rating: 4.7
---------------------------------
Name: PAON ALAS
Type: restaurant
Rating: 4.5
---------------------------------
Name: Rumah Makan Taman Sari
Type: restaurant
Rating: 4.3
---------------------------------
Name: KFC
Type: fast_food_restaurant
Rating: 4.1
---------------------------------
Name: Mutiara Resto Kintamani
Type: restaurant
Rating: 4.0
---------------------------------


In [52]:
import numpy as np
import tensorflow as tf

# Assuming place_features contains one-hot encoded types for each place
place_features = user_place_reviews[place_types].drop_duplicates().values
place_ids = user_place_reviews['id'].drop_duplicates().values

# Convert place_features to a TensorFlow constant
place_features_tf = tf.constant(place_features, dtype=tf.float32)

# Function to recommend places based on user-selected types using TensorFlow
def recommend_places_by_selected_types_tf(selected_types, top_n=5):
    # Convert user-selected types into a feature vector
    user_input = np.zeros(len(place_types))
    for stype in selected_types:
        if stype in place_types:
            user_input[place_types.index(stype)] = 1
    
    # Convert user_input to a TensorFlow constant
    user_input_tf = tf.constant(user_input.reshape(1, -1), dtype=tf.float32)  # Reshape to (1, len(place_types))
    
    # Model architecture
    input_layer = tf.keras.layers.Input(shape=(len(place_types),))
    embeddings = tf.keras.layers.Dense(len(place_types), activation='relu')(input_layer)
    similarities = tf.keras.layers.Dot(axes=1, normalize=True)([embeddings, user_input_tf])
    
    model = tf.keras.Model(inputs=input_layer, outputs=similarities)
    model.compile(optimizer='adam', loss='cosine_similarity')
    
    # Predict similarity between user input and place features
    similarities_pred = model.predict(place_features_tf).flatten()
    
    # Get indices of top places based on predicted similarities
    top_indices = np.argsort(similarities_pred)[-top_n:][::-1]
    
    # Get recommended place IDs
    recommended_place_ids = place_ids[top_indices]
    
    # Retrieve details of recommended places
    recommended_places = user_place_reviews[user_place_reviews['id'].isin(recommended_place_ids)]
    sorted_recommendations = recommended_places.sort_values(by='rating', ascending=False)
    sorted_recommendations = sorted_recommendations.drop_duplicates(subset=['name'])
    
    return sorted_recommendations[['name', 'primary-type', 'rating']]

# Example usage
selected_types = ['hotel', 'restaurant', 'cafe']  # Example user-selected types
recommendations = recommend_places_by_selected_types_tf(selected_types, top_n=10)
print("Top Recommendations based on Selected Types:")
print("---------------------------------")
for index, row in recommendations.iterrows():
    print(f"Name: {row['name']}")
    print(f"Type: {row['primary-type']}")
    print(f"Rating: {row['rating']}")
    print("---------------------------------")


22/22 [==============================] - 0s 4ms/step
Top Recommendations based on Selected Types:
---------------------------------
Name: BATUR BOAT TOUR AND WATER ACTIVITIES
Type: travel_agency
Rating: 4.9
---------------------------------
Name: Air Terjun Umejero
Type: tourist_attraction
Rating: 4.8
---------------------------------
Name: Bukit Catu Bungalows
Type: hotel
Rating: 4.8
---------------------------------
Name: Sumberkima Hill Retreat
Type: resort_hotel
Rating: 4.7
---------------------------------
Name: Studio L
Type: hotel
Rating: 4.6
---------------------------------
Name: Air Terjun Tembok Barak
Type: tourist_attraction
Rating: 4.5
---------------------------------
Name: Babi Guling Sunari
Type: indonesian_restaurant
Rating: 4.5
---------------------------------
Name: KopieNakal Menjangan
Type: restaurant
Rating: 4.4
---------------------------------
Name: Lovina Beach
Type: tourist_attraction
Rating: 4.3
---------------------------------
Name: Warung Nyoman
Type: indo

In [53]:
import numpy as np
import tensorflow as tf

# Assuming place_features contains one-hot encoded types for each place
place_features = user_place_reviews[place_types].drop_duplicates().values
place_ids = user_place_reviews['id'].drop_duplicates().values

# Function to recommend places based on user-selected types using TensorFlow
def recommend_places_by_selected_types_tf(selected_types, top_n=5):
    # Convert user-selected types into a feature vector
    user_input = np.zeros(len(place_types))
    for stype in selected_types:
        if stype in place_types:
            user_input[place_types.index(stype)] = 1
    
    # Convert user_input to a TensorFlow constant
    user_input_tf = tf.constant(user_input.reshape(1, -1), dtype=tf.float32)
    
    # Model architecture
    input_layer = tf.keras.layers.Input(shape=(len(place_types),))
    embeddings = tf.keras.layers.Dense(128, activation='relu')(input_layer)
    
    # User embedding for comparison
    user_embedding_layer = tf.keras.layers.Dense(128, activation='relu')(user_input_tf)
    
    similarities = tf.keras.layers.Dot(axes=-1, normalize=True)([embeddings, user_embedding_layer])
    
    model = tf.keras.Model(inputs=input_layer, outputs=similarities)
    model.compile(optimizer='adam', loss='cosine_similarity')
    
    # Predict similarity between user input and place features
    similarities_pred = model.predict(place_features).flatten()
    
    # Get indices of top places based on predicted similarities
    top_indices = np.argsort(similarities_pred)[-top_n:][::-1]
    
    # Get recommended place IDs
    recommended_place_ids = place_ids[top_indices]
    
    # Retrieve details of recommended places
    recommended_places = user_place_reviews[user_place_reviews['id'].isin(recommended_place_ids)]
    sorted_recommendations = recommended_places.sort_values(by='rating', ascending=False)
    sorted_recommendations = sorted_recommendations.drop_duplicates(subset=['name'])
    
    return sorted_recommendations[['name', 'primary-type', 'rating']]

# Example usage
selected_types = ['indonesian_restaurant', 'italian_restaurant', 'japanese_restaurant', 'jewelry_store',
    'korean_restaurant', 'lebanese_restaurant']  # Example user-selected types
recommendations = recommend_places_by_selected_types_tf(selected_types, top_n=10)
print("Top Recommendations based on Selected Types:")
print("---------------------------------")
for index, row in recommendations.iterrows():
    print(f"Name: {row['name']}")
    print(f"Type: {row['primary-type']}")
    print(f"Rating: {row['rating']}")
    print("---------------------------------")


22/22 [==============================] - 0s 5ms/step
Top Recommendations based on Selected Types:
---------------------------------
Name: UmahDeMadya Munduk & Rooftop De Madya Panoramic Restaurant
Type: guest_house
Rating: 4.9
---------------------------------
Name: Bukit Mende
Type: hindu_temple
Rating: 4.9
---------------------------------
Name: DAPUR PAPA EL
Type: cafe
Rating: 4.7
---------------------------------
Name: Gaia-Oasis Beach Resort
Type: resort_hotel
Rating: 4.7
---------------------------------
Name: Lanting Paras Resto And Pool... More than just a pool!
Type: swimming_pool
Rating: 4.6
---------------------------------
Name: ACK Fried Chicken Udayana
Type: fast_food_restaurant
Rating: 4.5
---------------------------------
Name: Warung Barokah
Type: indonesian_restaurant
Rating: 4.3
---------------------------------
Name: Sari Hill Bali
Type: restaurant
Rating: 4.2
---------------------------------
Name: My Lovina Hotel
Type: resort_hotel
Rating: 4.2
--------------------